In [1]:
# Import the dependencies.Numpy includes built-in random module and supplements.
import pandas as pd
import numpy as np

# Import the requests library.
import requests

# Import the datetime module from the datetime library.
from datetime import datetime

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.(unzip)
coordinates = list(lat_lngs)

# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [3]:
# Create an empty list to hold the weather data.
city_data = []

# Create counters.
record_count = 1
set_count = 1

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
    
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Add 1 to the record count.
    i += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = (city_weather["coord"]["lat"])
        city_lng = (city_weather["coord"]["lon"])
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country":city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
    except KeyError:
        continue
len(city_data)

683

In [4]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Meulaboh,ID,4.1363,96.1285,84.06,64,27,3.13,scattered clouds
1,Trairi,BR,-3.2778,-39.2689,76.66,90,100,4.52,overcast clouds
2,Olafsvik,IS,64.8945,-23.7142,28.94,74,100,13.87,overcast clouds
3,Punta Arenas,CL,-53.1500,-70.9167,44.60,61,0,4.61,clear sky
4,Cape Town,ZA,-33.9258,18.4232,66.20,77,75,9.22,broken clouds
5,Hermanus,ZA,-34.4187,19.2345,63.00,94,99,5.19,overcast clouds
6,Butaritari,KI,3.0707,172.7902,80.91,80,85,11.83,overcast clouds
7,Hobart,AU,-42.8794,147.3294,64.00,29,0,10.36,clear sky
8,Busselton,AU,-33.6500,115.3333,71.01,50,0,5.01,clear sky
9,Mahebourg,MU,-20.4081,57.7000,80.60,83,40,9.22,light rain


In [5]:
# Create the output file (CSV).
output_data_file = "../Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")